- Realización del modelo que predice el precio de un diamante en base a sus características

- importamos la librerías que se utilizaran para realizar el código a continuación se comentara cada linea para poder entender el procesos que se realiza 

In [1]:
# importamos pandas para leer y manipular los datos estructurados en formato CSV
import pandas as pd

# importamos utilidades para dividir los datos en entrenamiento y prueba, y para hacer búsqueda de hiperparámetros y validación cruzada
from sklearn.model_selection import train_test_split, GridSearchCV

# importamos OrdinalEncoder para convertir variables categóricas con jerarquía lógica en números enteros ordenados
from sklearn.preprocessing import OrdinalEncoder

# importamos Pipeline para encadenar pasos de preprocesamiento y modelado en un solo flujo automatizado
from sklearn.pipeline import Pipeline

# importamos ColumnTransformer para aplicar distintos transformadores a distintas columnas del dataset
from sklearn.compose import ColumnTransformer

# importamos el modelo XGBRegressor del paquete xgboost, especializado en regresión basada en boosting
from xgboost import XGBRegressor

# importamos root_mean_squared_error como métrica de evaluación del modelo, útil para cuantificar el error promedio de las predicciones
from sklearn.metrics import root_mean_squared_error

- leemos el csv para manipular los datos que le pasaremos al modelo


In [418]:
df = pd.read_csv("diamonds.csv") 
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335


- eliminamos las filas que son exactamente iguales 

In [419]:
df.drop_duplicates(inplace=True)

- definimos la jerarquía de las columnas categóricas por que unos son de mayor valor que otros 

In [420]:
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']


- codificamos los valores numéricos respetando el orden que definimos antes en las listan aterieres

In [421]:
ordinal_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

- creamos las variables para realizar posteriormente las division de datos de entrenamiento 

In [422]:
X = df.drop("price", axis=1)
Y = df["price"]

- definimos qué variables del conjunto de datos son categóricas y cuáles son numéricas

In [423]:
categorical_cols = ['cut', 'color', 'clarity']
numerical_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']

- realizamos la trasformación de las columnas y las que no están definidas en categorical_cols estarán tal cual como están

In [424]:
preprocessor = ColumnTransformer(transformers=[
    ('cat', ordinal_encoder, categorical_cols)
], remainder='passthrough')

- crea una instancia del modelo XGBoost para regresión configurado para minimizar el error cuadrático medio


In [425]:
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

- usamos las clase Pipeline para definir el pre procesamiento para hacer uso de modelo en base a las regresión lineal

In [426]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', xgb)
])

- definimos las combinaciones de parámetros para el modelo

In [427]:
param_grid = {
    'regressor__n_estimators': [100, 230], # cantidad de arboles
    'regressor__max_depth': [4, 6, 8], # profundidad de encontrar patrones 
    'regressor__learning_rate': [0.05, 0.1, 0.2] # ajustes de para los arboles( mientras mas chico mas arboles requerirá ) 
}


- aplicamos al modelo los parámetros para que valida las mejor combinación de parámetros 

In [428]:
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)

- probando todas las combinaciones de hiperparámetros definidas en param_grid para el pipeline que contiene tu preprocesador y el XGBRegressor.


In [429]:
grid_search.fit(X, Y)
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 18 candidates, totalling 54 fits


- dividimos los datos. el 80% en X_train, Y_train y el 20% a X_test, Y_test
- entrenamos el modelo pasando los datos X_train y Y_train

In [430]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)


,steps,"[('preprocessing', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


- hacemos una predicción para realizar validación

In [431]:
y_pred = best_model.predict(X_test)

- calculamos error promedio entre las predicciones y los valores reales, pero en las mismas unidades que el precio

In [432]:
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Mejor RMSE en test: {rmse:.2f} USD")
print("Mejores parámetros encontrados:", grid_search.best_params_)


Mejor RMSE en test: 507.36 USD
Mejores parámetros encontrados: {'regressor__learning_rate': 0.2, 'regressor__max_depth': 4, 'regressor__n_estimators': 230}


- realizamos una prueba con datos de la primera fila del datase pero sin el precio aver cuanto nos da 

In [433]:
nuevo_diamante_row_1 = pd.DataFrame([{
  'carat': 0.23,
    'cut': 'Ideal',
    'color': 'E',
    'clarity': 'SI2',
    'depth': 61.5,
    'table': 55.0,
    'x': 3.95,
    'y': 3.98,
    'z': 2.43

}])

- hacemos la predicción con el dato del dataset y aver cuantos estima su resultado 

In [439]:
prediction = best_model.predict(nuevo_diamante_row_1)
print(f"Precio estimado: ${prediction[0]:.2f} USD")

Precio estimado: $320.36 USD


- exportamos el modelo para saber si se realizo correctamente para usarlo donde queramos

In [435]:
from joblib import dump,load
dump(best_model,'modeloDiamonds.joblib')

['modeloDiamonds.joblib']

- lo cargamos y vemos con los datos anteriores de la primera fila del dataset si nos devuelve lo mismo  

In [436]:
model_load = load('modeloDiamonds.joblib')

In [438]:
nuevo_diamante_max = pd.DataFrame([{
    'carat': 2.29,
    'cut': 'Premium',
    'color': 'I',
    'clarity': 'VS2',
    'depth': 60.8,
    'table': 60.0,
    'x': 8.50,
    'y': 8.47,
    'z': 5.16
}])

# estimación 18823


# Predecir con el modelo cargado
prediccion = model_load.predict(nuevo_diamante_max)
print(f"Precio estimado: ${prediccion[0]:.2f} USD")

Precio estimado: $17723.80 USD


## modelo que estima el valor de un diamante en base a sus características realizado correctamente con un modelo de RandomForest